# Reaching multiple targets with a manipulator
The objective of this exercise is to reach multiple targets with a manipulator.

We provide a basic example for reaching one point, and you have to modify it for sequence of multiple targets. Below it is the basic example, there we'll guide you to the final result.


## Set up

In [ ]:
import magic_donotload

We will need crocoddyl as in the previous notebook, with the model of the arm of the humanoid robot Talos, a 7-dof arm. It can be found in example robot data.

In [ ]:
import crocoddyl
import pinocchio
import numpy as np
import example_robot_data as robex

## The optimal-control program

First, let's load the Pinocchio model for the Talos arm.

In [ ]:
robot = robex.load('talos_arm')
robot_model = robot.model

Set robot model.

In [ ]:
robot_model.armature =np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.])*5
robot_model.q0 = np.array([3.5,2,2,0,0,0,0])
robot_model.x0 = np.concatenate([robot_model.q0, np.zeros(robot_model.nv)])
robot_model.gravity *= 0

Let's add a viewer to display the model.


In [ ]:
from utils.meshcat_viewer_wrapper import MeshcatVisualizer
viz = MeshcatVisualizer(robot)
viz.display(robot_model.q0)
viz.viewer.jupyter_cell()

In [ ]:
viz.addBox('world/box',[.1,.1,.1], [1.,0,0,1])
viz.addBox('world/goal',[.1,.1,.1],[0,1,0,1])
viz.applyConfiguration('world/goal',[0.2,0.5,.5,0,0,0,1])

Configure the task.

In [ ]:
FRAME_TIP = robot_model.getFrameId("gripper_left_fingertip_3_link")
goal = np.array([.2,0.5,.5])

Create a cost model per the running and terminal action model.

In [ ]:
state = crocoddyl.StateMultibody(robot_model)
runningCostModel = crocoddyl.CostModelSum(state)
terminalCostModel = crocoddyl.CostModelSum(state)

Note that we need to include a cost model (i.e. set of cost functions) in order to fully define the action model for our optimal control problem.
For this particular example, we formulate three running-cost functions: goal-tracking cost, state and control regularization; and a terminal cost: goal cost. First, let's create the common cost functions.

In [ ]:
### Cost for reaching the target
Mref = crocoddyl.FramePlacement(FRAME_TIP,pinocchio.SE3(np.eye(3), goal))
pref = crocoddyl.FrameTranslation(FRAME_TIP,goal)
goalTrackingCost = crocoddyl.CostModelFrameTranslation(state, pref)
weights=crocoddyl.ActivationModelWeightedQuad(np.array([1,1,1,1,1,1,1, 1,1,1,1,2,2,2.]))

### Cost for regularizing the state about robot_model.x0
xRegCost = crocoddyl.CostModelState(state,weights,robot_model.x0)
weightsT=crocoddyl.ActivationModelWeightedQuad(np.array([.01,.01,.01,.01,.01,.01,.01, 1,1,1,1,2,2,2.]))
xRegCostT = crocoddyl.CostModelState(state,weightsT,robot_model.x0)

### Cost for keeping the control low
uRegCost = crocoddyl.CostModelControl(state)


Then let's added the running and terminal cost functions

In [ ]:
runningCostModel.addCost("gripperPose", goalTrackingCost, .001)
runningCostModel.addCost("xReg", xRegCost, 1e-3)
runningCostModel.addCost("uReg", uRegCost, 1e-6)
terminalCostModel.addCost("gripperPose", goalTrackingCost, 10)
terminalCostModel.addCost("xReg", xRegCostT, .01)

Next, we need to create an action model for running and terminal knots. The
forward dynamics (computed using ABA) are implemented inside DifferentialActionModelFullyActuated.

In [ ]:
actuationModel = crocoddyl.ActuationModelFull(state)
dt = 1e-2
runningModel = crocoddyl.IntegratedActionModelEuler(
    crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuationModel, runningCostModel), dt)
runningModel.differential.armature = robot_model.armature
terminalModel = crocoddyl.IntegratedActionModelEuler(
    crocoddyl.DifferentialActionModelFreeFwdDynamics(state, actuationModel, terminalCostModel), 0.)
terminalModel.differential.armature = robot_model.armature

For this optimal control problem, we define 250 knots (or running action
models) plus a terminal knot

In [ ]:
T = 100
problem = crocoddyl.ShootingProblem(robot_model.x0, [runningModel] * T, terminalModel)

We finalize the set up by creating the DDP solver for this optimal control problem.

In [ ]:
ddp = crocoddyl.SolverDDP(problem)

Callbacks can be added, for example to at verbose output when the solver runs, or logs the data of the descent algorithm.

In [ ]:
ddp.setCallbacks([
    crocoddyl.CallbackLogger(),
    crocoddyl.CallbackVerbose(),
])

The DDP algorithm is run by the solver with:

In [ ]:
ddp.solve([],[],1000)  # xs_init,us_init,maxiter

You can now look at the results, either in Gepetto-viewer by running the trajectory, or by plotting it.

In [ ]:
# %load -r 101 examples/arm_example.py
import utils.croco_utils as crocutils

And plotting...

In [ ]:
%matplotlib inline

In [ ]:
# %load -r 97-99 tp5/arm_example.py
log = ddp.getCallbacks()[0]
crocoddyl.plotOCSolution(log.xs, log.us, figIndex=1)

Some elements describing the solver convergence can also been plotted.

In [ ]:
crocoddyl.plotConvergence(log.costs, log.u_regs, log.x_regs, log.grads, log.stops, log.steps, figIndex=2)

## I. DifferentialActionModel for Pinocchio ABA
This scenario uses an action model that computes 2nd order differential dynamics with Pinocchio. Note that it can accept several cost models. This action model is tailored for robot applications, and at the same time, it's modular since:
 - you can modify the robot dynamics by changing Pinocchio model, and
 - you can formulate any cost function by simply adding running a terminal costs.

## II. Cost models

A cost model computes a scalar cost value and its gradient and Hessian. All the models implemented are computing a cost residual and are computing the Hessian with the Gauss approximation.

We implemented reusable cost models for controlling 
 - a frame placement (translation or velocity),
 - the center of mass position, and 
 - state  and control spaces.

In the example above, we used the CostModelFrameTranslation which defines a 3d position task, and the state and control regularizers.

All functions are defined by their *model*, which is const, i.e none of its fields is modified by the *calc* and *calcDiff* calls. Temporary buffers are defined in a companion *data*.

In [ ]:
dataCollector = crocoddyl.DataCollectorMultibody(robot.data)
trackData = goalTrackingCost.createData(dataCollector)

### II.a Frame position cost

You define a frame ID and the reference position as a 3D array. The cost is the distance between the frame and the target. This cost depends on $\mathbf{x}$ (specifically the configuration $\mathbf{q}$). You can double check the 0s in its gradient.

In [ ]:
x = ddp.xs[1].copy()
q = x[:state.nq]
pinocchio.updateFramePlacements(robot.model, robot.data)
pinocchio.computeJointJacobians(robot.model, robot.data, q)
goalTrackingCost.calc(trackData, x)
goalTrackingCost.calcDiff(trackData, x)
print('Lx = ',trackData.Lx)
print('Lu = ',trackData.Lu)

### II.b State cost
In this part of the tutorial you must define a State model. It defines 
 - the dimension of the state and its tangent, and
 - the exponential/integrate and difference/log operators.
The operators can described using Pinocchio functions. And the exercite consists on adding them into your State class. Please note crocoddyl has abstract functions for this.

The state cost uses a reference in state space (State.zero() by default). The cost is the distance, computed with state.difference between the current state and the reference. Hence, with this cost, we regularize both position and velocity.

### II.c Control cost

The control cost uses a control reference as in the state cost. The cost is the distance the current control and the reference. Hence the cost regularizes torque commands.

### II.d Add cost models to the differential action model
Each time we want to include a new cost function, we use addCost function inside our DAM. In this function you're also able its weight.

## III. Create the problem with integrated action model
Differential action models describe cost and dynamics in continuous-time, however our optimal control solvers work in discrete-time. We have created the integrated action model in order to deal with this.

In the previous code, we have used an abstract class that uses simpletic Euler rules. In the cartpole exercise you have learnt how to use integrated action models for your problem.

## IV. Callbacks

Callback functions are needed for analysing and debugging the performance of the solver for your specific problem.
For problems defined with Pinocchio, you can display the robot trajectory per each iterate by including CallbackDisplay. With this callback, you can display robot motions with different rates. Additionally, CallbackVerbose prints a message that allows us to understand the behaviour of the solver.

Generally speaking, an user is able to describe any callback function. This function will be run once per iterate and it has access to all data.

## VI. Modifying the example

Start by defining several targets (let's say 4 targets, all at x=0.4, and at y and z being either 0 or 0.4), and display then in the viewer.


The shooting problem will be composed of 4 sequences of action models. Each sequence consists on T shooting "running" nodes and 1 terminal node. The running nodes mostly have regularization terms, while the terminal nodes have a strong cost toward the respective target.

[ R1,R1,R1 ... R1,T1, R2,R2 .... R2, T2, R3 ... R3, T3, R4 ... R4 ] , T4

First create 4 running models and 4 terminal models.

Then you need to add a position cost, and state and control regularization to each running action model. Please  note that for terminal action model is only needed the position cost. Additionally, in the running models, the position cost should be low, and it should be high in the terminal models.

Now create a shooting problem.

In [ ]:
seq0 = [runningModels[0]]*T + [terminalModels[0]]
seq1 = [runningModels[1]]*T + [terminalModels[1]]
seq2 = [runningModels[2]]*T + [terminalModels[2]]
seq3 = [runningModels[3]]*T 
problem = crocoddyl.ShootingProblem(x0,seq0+seq1+seq2+seq3,terminalmodel[3])

Create a DDP solver for this problem and run it. 

In [ ]:
ddp = crocoddyl.SolverDDP(problem)
ddp.solve()

Well, it should not work, at least no on the first shot. The DDP solver is likely not strong enough to accept the random weights that you have selected. 

If it is working nicely from the first shot, display it in the viewer and go take a coffee. But you will likely have to tweak the gains to make it work.

**It is suggested to first optimize only sequence 1. When you are happy with it, add sequence 2 and optimize again, etc.**


## V. Penalty
The solver works with double precisions, so it is quite robust to high weight. 10000 is likely to be accepted for example. But if you make the problem too difficult, the solver will break. 

In that case, you can implement a simple penalty solver by setting the weight to be 10**i, and creating a for loop to explore i from 0 to 5. At each iteration of the loop, run the solver from the previous solution and for few iterations only.

In [ ]:
for i in range(1,6):
    for m in terminalModels:
        m.costs.costs['gripperPose'].weight = 10**i
    ddp.solve(ddp.xs, ddp.us, 10)